In [1]:
import os 
os.chdir('/Users/egill/dev/genki/genki-signals')

from genki_signals.buffers import DataBuffer

buffer = DataBuffer(maxlen=400)

In [43]:
from genki_signals.signal_sources import Sampler, RandomNoise, SineWave
import numpy as np

sine_2 = SineWave(amplitude=8, frequency=7, phase=0)

source = Sampler(
    {
        'sine_1': lambda t: 5 * np.sin(2 * np.pi * (3/2) * t),
        'sine_2': sine_2,
        'noise': RandomNoise(),
    },
    sample_rate=100)

In [44]:
import time
from threading import Thread

def run_source(source, buffer):
    source.start()
    while source.is_active:
        buffer.extend(source.read())
        time.sleep(1 / 25)
        


In [45]:
t = Thread(target=run_source, args=(source, buffer))
t.start()

In [46]:
buffer.plot('sine_1')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='sine_1', orientation='vertical', scale=LinearSc…

In [47]:
from ipywidgets import interact, IntSlider, FloatSlider

@interact(amplitude=FloatSlider(min=1, max=5, step=0.5, value=5), frequency=IntSlider(min=1, max=10, value=3))
def set_wave_params(amplitude, frequency):
    sine_2.amplitude = amplitude
    sine_2.frequency = frequency

interactive(children=(FloatSlider(value=5.0, description='amplitude', max=5.0, min=1.0, step=0.5), IntSlider(v…

In [49]:
source.stop(); t.join()

In [50]:
import genki_signals.signal_functions as sfions as s
from genki_signals.signal_system import SignalSystem

derived = [
    sf.Sum('sine_1', 'sine_2', name='sines_sum'),
    sf.Sum('sines_sum', 'noise', name='wave_with_noise'),
    sf.FourierTransform('wave_with_noise', name='spectrum', window_size=128, window_overlap=64)
]

buffer = DataBuffer(maxlen=400)
system = SignalSystem(source, derived)

In [51]:
t = Thread(target=run_source, args=(system, buffer))
t.start()

In [52]:
buffer

DataBuffer(max_size=400, data=timestamp: (75,)
sine_1: (75,)
sine_2: (75,)
noise: (75,)
sines_sum: (75,)
wave_with_noise: (75,)
spectrum: (65, 0))

In [53]:
buffer.plot('wave_with_noise')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='wave_with_noise', orientation='vertical', scale…

In [54]:
buffer.plot('spectrum', plot_type='spectrogram', window_size=128, sample_rate=100)

Figure(axes=[Axis(label='Hz', scale=LinearScale()), Axis(label='db', orientation='vertical', scale=LinearScale…

In [58]:
system.stop(); t.join()

In [30]:
from genki_signals.signal_sources import WaveDataSource

wave = WaveDataSource(ble_address='7CEF8BE3-F544-99C9-B25A-3C6DD1281491')

buffer = DataBuffer(maxlen=400)
system = SignalSystem(wave, [])

In [31]:
t = Thread(target=run_source, args=(system, buffer))
t.start()

Connecting to wave at address 7CEF8BE3-F544-99C9-B25A-3C6DD1281491
Connected to Wave


In [33]:
buffer.plot('gyro')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='gyro', orientation='vertical', scale=LinearScal…